# Linear Regression Model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from IPython.display import clear_output

2022-08-11 16:42:08.222416: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-11 16:42:08.222453: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Import

In [2]:
# Load dataset.
dftrain = pd.read_csv('/mnt/MyBoX/Programming/TensorFlow 2.0/Notebooks/Data Sets/Titanic/train.csv') # training data
dftest = pd.read_csv('/mnt/MyBoX/Programming/TensorFlow 2.0/Notebooks/Data Sets/Titanic/eval.csv') # testing data
y_train = dftrain.pop('survived')
y_test = dftest.pop('survived')


### Categorical and Numeric data


In [3]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()  # gets a list of all unique values from given feature column
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

### Input Function

In [4]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use

train_input_fn = make_input_fn(dftrain, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
test_input_fn = make_input_fn(dftest, y_test, num_epochs=1, shuffle=False)
 

### Model Creation

In [5]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
# We create a linear estimtor by passing the feature columns we created earlier
clear_output()

### Training

In [6]:
linear_est.train(train_input_fn)  # train
clear_output()  # clears consoke output

### Testing

In [7]:
result = linear_est.evaluate(test_input_fn)  # get model metrics/stats by testing on tetsing data
clear_output()  # clears consoke output

In [8]:
print(result)  # the result variable is simply a dict of stats about our model

{'accuracy': 0.77272725, 'accuracy_baseline': 0.625, 'auc': 0.8251913, 'auc_precision_recall': 0.79661065, 'average_loss': 0.49326497, 'label/mean': 0.375, 'loss': 0.48479244, 'precision': 0.7294118, 'prediction/mean': 0.34564358, 'recall': 0.6262626, 'global_step': 200}


### Prediction

In [9]:
predict= list(linear_est.predict(test_input_fn))
clear_output()

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmprg12cse8/model.ckpt-200
INFO:tensorflow:Running local_init_op.


2022-08-11 16:42:41.891800: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-11 16:42:41.892131: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


INFO:tensorflow:Done running local_init_op.


In [10]:
print(dftest.loc[0])
print("Survival Probability : ", predict[0]['probabilities'][1])
print("Survive : ", y_test.loc[0])


sex                          male
age                          35.0
n_siblings_spouses              0
parch                           0
fare                         8.05
class                       Third
deck                      unknown
embark_town           Southampton
alone                           y
Name: 0, dtype: object
Survival Probability :  0.08409417
Survive :  0
